# Setup Piz Daint with AiiDA

In [ ]:
%%html
<img width="400px" src="http://www.cscs.ch/fileadmin/uploads/CSCS_PIZDAINT_0090_01.jpg"><br>
For more information visit <a target="_blank" href="http://www.cscs.ch/computers/piz_daint/index.html">cscs.ch</a>

In [ ]:
from __future__ import print_function

from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)

from aiida.orm.querybuilder import QueryBuilder
from aiida.orm import load_node, Code, Computer
from aiida.common.exceptions import NotExistent
from aiida.backends.utils import get_automatic_user
#from aiida.backends.sqlalchemy.models.authinfo import DbAuthInfo
from aiida.backends.djsite.db.models import DbAuthInfo

import ipywidgets as ipw
from IPython.display import display, clear_output

from subprocess import check_call, check_output, call
from os import path
import re

In [ ]:
layout = ipw.Layout(width="400px")
style = {"description_width":"150px"}

## Step 1: Setup ssh

In [ ]:
def ssh_keygen():
    fn = path.expanduser("~/.ssh/id_rsa")
    if not path.exists(fn):
        print("Creating ssh key pair")
        !  mkdir -p /project/.ssh
        ! ssh-keygen -f {fn} -t rsa -N ''
    
def is_host_known(hostname):
    fn = path.expanduser("~/.ssh/known_hosts")
    if not path.exists(fn):
        return False
    return call(["ssh-keygen", "-F", hostname]) == 0

def make_host_known(hostname, proxycmd=[]):
    fn = path.expanduser("~/.ssh/known_hosts")
    print("Adding keys from %s to %s"%(hostname, fn))
    hashes = check_output(proxycmd+["ssh-keyscan", "-H", hostname])
    with open(fn, "a") as f:
        f.write(hashes)

def can_login(hostname, username):
    userhost = username+"@"+hostname
    print("Trying ssh "+userhost+"... ", end='')
    ret = call(["ssh", userhost, "true"])
    print("Ok" if ret==0 else "Failed")
    return ret==0

def print_pubkey(hostname, username):
    fn = path.expanduser("~/.ssh/id_rsa.pub")
    key = open(fn).read().strip()
    print('ssh '+username+'@'+hostname+' \'echo "'+key+'" >> ~/.ssh/authorized_keys\'')

def is_in_config(hostname):
    fn = path.expanduser("~/.ssh/config")
    if not path.exists(fn):
        return False
    cfglines = open(fn).read().split("\n")
    return "Host "+hostname in cfglines

def write_ssh_config(targethost, username, proxyhost):
    fn = path.expanduser("~/.ssh/config")
    print("Adding section to "+fn)
    with open(fn, "a") as f:
        f.write("Host "+targethost+"\n")
        f.write("User "+username+"\n")
        f.write("ProxyCommand ssh "+username+"@"+proxyhost+" netcat %h %p\n")
        
def fetch_slurm_accounts(hostname, username):
    userhost = username+"@"+hostname
    print("Fetching slurm accounts")
    
    sbucheck = check_output(["ssh", username+"@daint.cscs.ch", "sbucheck"])
    slurm_accounts = re.findall("\* (\w+):.*constraints: (\w+)", sbucheck)
    options = [("Please select a slurm account", False)]
    for l, c in slurm_accounts:
        options.append(["%s (%s)"%(l,c), (l,c)])
    drop_account.options = options

def setup_ssh(proxyhost, targethost, username):
    ssh_keygen()
    
    # setup proxy first
    if not is_host_known(proxyhost):
        make_host_known(proxyhost)

    if not can_login(proxyhost, username):
        print("Please execute the following command and then try again:\n")
        print_pubkey(proxyhost, username)
        return False
                
    # now setup target host
    if not is_host_known(targethost):
        make_host_known(targethost, proxycmd=['ssh', username+"@"+proxyhost])
    
    if not is_in_config(targethost):
        write_ssh_config(targethost, username, proxyhost)
         
    # final check
    if can_login(targethost, username):
        fetch_slurm_accounts(targethost, username)
        print("Automatic ssh setup successful :-)")
        return True
    else:
        print("Automatic ssh setup failed, sorry :-(")
        return False

In [ ]:
def on_setup_ssh(b):
    with setup_ssh_out:
        clear_output()
        if len(inp_username.value.strip())==0:
            print("Please enter your CSCS username")
            return
        setup_ssh("ela.cscs.ch", "daint.cscs.ch", inp_username.value)

inp_username = ipw.Text(description="CSCS user name:", layout=layout, style=style)
btn_setup_ssh = ipw.Button(description="Setup ssh")
btn_setup_ssh.on_click(on_setup_ssh)
setup_ssh_out = ipw.Output()

display(inp_username,btn_setup_ssh, setup_ssh_out)

## Step 2: Setup AiiDA Computer

In [ ]:
# https://github.com/aiidateam/aiida_core/blob/develop/aiida/cmdline/commands/computer.py#L400

def setup_computer(computer_name, username, account, partition):
    try:
        computer = Computer.get(computer_name)
        print("A computer called {} already exists.".format(computer_name))
        return
    except NotExistent:
        pass

    print("Creating new computer with name '{}'".format(computer_name))
    computer = Computer(name=computer_name)
    computer.set_hostname("daint.cscs.ch")
    computer.set_description("Named after Piz Daint, a prominent peak in Grisons that overlooks the Fuorn pass, this supercomputer is a hybrid Cray XC50/XC40 system and is the flagship system for national HPC Service.")
    computer.set_enabled_state(True)
    computer.set_transport_type("ssh")
    computer.set_scheduler_type("slurm")
    computer.set_workdir("/scratch/snx3000/"+username+"/aiida_run_"+computer_name)
    # set_mpirun_command() must be called after set_scheduler_type()
    #cmd = "srun -n {tot_num_mpiprocs} -c $SLURM_CPUS_PER_TASK --cpu_bind=rank --hint=nomultithread"
    cmd = "srun -n {tot_num_mpiprocs}"
    computer.set_mpirun_command(cmd.split())
    ncpus = 12 if partition == "gpu" else 36
    computer.set_default_mpiprocs_per_machine(ncpus)
    prepend_text  = "### computer prepend_text start ###\n"
    prepend_text += "#SBATCH --account=%s\n"%account
    prepend_text += "#SBATCH --constraint=%s\n"%partition
    prepend_text += "#SBATCH --cpus-per-task=1\n"
    prepend_text += "export OMP_NUM_THREADS=$SLURM_CPUS_PER_TASK\n"
    prepend_text += "source $MODULESHOME/init/bash\n"
    prepend_text += "module load daint-%s\n"%partition
    prepend_text += "ulimit -s unlimited\n"
    if partition == "gpu":
        prepend_text += "export CRAY_CUDA_MPS=1\n"
    prepend_text += "### computer prepend_text end ###\n"
    computer.set_prepend_text(prepend_text)
    computer.store()
    
    # create DbAuthInfo
    authparams = {
        'compress': True,
        'gss_auth': False,
        'gss_deleg_creds': False,
        'gss_host': 'daint.cscs.ch',
        'gss_kex': False,
        'key_policy': 'RejectPolicy',
        'load_system_host_keys': True,
        'port': 22,
        'proxy_command': 'ssh '+username+'@ela.cscs.ch netcat daint.cscs.ch 22',
        'timeout': 60,
        'username': username,
    }
    aiidauser = get_automatic_user()
    authinfo = DbAuthInfo(dbcomputer=computer.dbcomputer, aiidauser=aiidauser)
    authinfo.set_auth_params(authparams)
    authinfo.save()
    
    ! verdi computer show {computer_name}

In [ ]:
def on_account_change(c):
    if drop_account.value:
        inp_compname.value = inp_compname.value.rsplit("-",1)[0] + "-" + drop_account.value[0]

def on_setup_computer(b):
    with setup_comp_out:
        clear_output()
        if len(inp_compname.value.strip())==0:
            print("Please enter computer name")
            return
        if not drop_account.value:
            print("Plese select a slurm account")
            return
        account, partition = drop_account.value
        setup_computer(inp_compname.value, inp_username.value, account, partition)

    
inp_compname = ipw.Text(description="Computer name:", value="daint", layout=layout, style=style)
drop_account = ipw.Dropdown(description="Slurm Account:", options={"first run ssh setup":False}, style=style, layout=layout )
drop_account.observe(on_account_change, names='value')
btn_setup_comp = ipw.Button(description="Setup Computer")
btn_setup_comp.on_click(on_setup_computer)
setup_comp_out = ipw.Output()
display(inp_compname, drop_account, btn_setup_comp, setup_comp_out)

## Step 3: Test Computer

In [ ]:
def on_test_computer(b):
    with test_out:
        clear_output()
        if len(inp_compname.value.strip())==0:
            print("Please enter computer name")
            return
        ! verdi computer test {inp_compname.value}

test_out = ipw.Output()
btn_test_comp = ipw.Button(description="Test Computer")
btn_test_comp.on_click(on_test_computer)
display(btn_test_comp, test_out)

## Step 4: Setup codes

In [ ]:
def setup_codes(computer_name, partition):
    computer = Computer.get(computer_name)
    
    if partition == "mc":
        qe_path = "/apps/daint/UES/jenkins/6.0.UP04/mc/easybuild/software/QuantumESPRESSO/6.1.0-CrayIntel-17.08/bin/"
        cp2k_path = "/apps/daint/UES/jenkins/6.0.UP04/mc/easybuild/software/CP2K/5.0r18043-CrayGNU-17.08/bin/"
    elif partition == "gpu":
        qe_path = "/apps/daint/UES/jenkins/6.0.UP04/gpu/easybuild/software/QuantumESPRESSO/6.1.0-CrayIntel-17.08/bin/"
        cp2k_path = "/apps/daint/UES/jenkins/6.0.UP04/gpu/easybuild/software/CP2K/5.0r18043-CrayGNU-17.08-cuda-8.0/bin/"
    else:
        raise("unkown partition "+partition)
    
    # CP2K
    code = Code(remote_computer_exec=(computer, cp2k_path + "cp2k.psmp"))
    code.label = "cp2k"
    code.description = "CP2K Code"
    code.set_input_plugin_name("cp2k")
    prepend_text  = "### code prepend_text start ###\n"
    prepend_text += "module load CP2K\n"
    prepend_text += "export CP2K_DATA_DIR=/users/schuetto/git/cp2k/cp2k/data/\n"
    prepend_text += "### code prepend_text end ###\n"
    code.set_prepend_text(prepend_text)
    code.set_append_text("")
    code._reveal()
    code.store()
    ! verdi code show "cp2k@{computer_name}"
    
    # QuantumESPRESSO pw
    code = Code(remote_computer_exec=(computer, qe_path + "pw.x"))
    code.label = "pw"
    code.description = "QuantumESPRESSO pw"
    code.set_input_plugin_name("quantumespresso.pw")
    prepend_text  = "### code prepend_text start ###\n"
    prepend_text += "module load QuantumESPRESSO/6.1.0-CrayIntel-17.08\n"
    prepend_text += "### code prepend_text end ###\n"
    code.set_prepend_text(prepend_text)
    code.set_append_text("")
    code._reveal()
    code.store()
    ! verdi code show "pw@{computer_name}"
    
    # QuantumESPRESSO pp
    code = Code(remote_computer_exec=(computer, qe_path + "pp.x"))
    code.label = "pp"
    code.description = "QuantumESPRESSO pp"
    code.set_input_plugin_name("quantumespresso.pp")
    prepend_text  = "### code prepend_text start ###\n"
    prepend_text += "module load QuantumESPRESSO/6.1.0-CrayIntel-17.08\n"
    prepend_text += "### code prepend_text end ###\n"
    code.set_prepend_text(prepend_text)
    code.set_append_text("")
    code._reveal()
    code.store()
    ! verdi code show "pp@{computer_name}"
    
    # QuantumESPRESSO projwfc
    code = Code(remote_computer_exec=(computer, qe_path + "projwfc.x"))
    code.label = "projwfc"
    code.description = "QuantumESPRESSO projwfc"
    code.set_input_plugin_name("quantumespresso.projwfc")
    prepend_text  = "### code prepend_text start ###\n"
    prepend_text += "module load QuantumESPRESSO/6.1.0-CrayIntel-17.08\n"
    prepend_text += "### code prepend_text end ###\n"
    code.set_prepend_text(prepend_text)
    code.set_append_text("")
    code._reveal()
    code.store()
    ! verdi code show "projwfc@{computer_name}"

In [ ]:
def on_setup_codes(b):
    with setup_code_out:
        clear_output()
        partition = drop_account.value[1]
        setup_codes(inp_compname.value, partition)

setup_code_out = ipw.Output()
btn_setup_codes = ipw.Button(description="Setup Codes")
btn_setup_codes.on_click(on_setup_codes)
display(btn_setup_codes, setup_code_out)